# Codificación de variables categóricas

En este notebook, presentamos algunas formas típicas de tratar con **variables
categóricas** codificándolas, como la **codificación ordinal** y la
**codificación one-hot**.


Primero carguemos el conjunto de datos "adult" que contiene tanto datos numéricos como categóricos.


In [2]:
import pandas as pd

adult_census = pd.read_csv("adult.csv")
# drop the duplicated column `"education-num"` as stated in the first notebook
adult_census = adult_census.drop(columns="education.num")

target_name = "income"
target = adult_census[target_name]

data = adult_census.drop(columns=[target_name])


## Identificar variables categóricas

Como vimos en la sección anterior, una variable numérica es una cantidad
representada por un número real o entero. Estas variables pueden ser
manejadas de manera natural por algoritmos de aprendizaje automático que
suelen estar compuestos por una secuencia de instrucciones aritméticas como
sumas y multiplicaciones.

En cambio, las variables categóricas tienen valores discretos,
típicamente representados por etiquetas de texto (aunque no siempre) tomadas
de una lista finita de opciones posibles. Por ejemplo, la variable `native-country`
en nuestro conjunto de datos es una variable categórica porque codifica los
datos utilizando una lista finita de posibles países (junto con el símbolo `?`
cuando falta esta información):


In [3]:
data["native.country"].value_counts().sort_index()

,count
native.country,
?,583
Cambodia,19
Canada,121
China,75
Columbia,59
Cuba,95
Dominican-Republic,70
Ecuador,28
El-Salvador,106


¿Cómo podemos reconocer fácilmente las columnas categóricas en el conjunto de datos? Parte de la respuesta radica en el tipo de datos de las columnas:

In [4]:
data.dtypes

,0
age,int64
workclass,object
fnlwgt,int64
education,object
marital.status,object
occupation,object
relationship,object
race,object
sex,object
capital.gain,int64


Si observamos la columna `"native.country"`, vemos que su tipo de datos es `object`, lo que significa que contiene valores de texto.

## Seleccionar características según su tipo de datos

Anteriormente hemos definimos manualmente las columnas numéricas. Podríamos hacer un enfoque similar. Pero ahora vamos a utilizar la función auxiliar de scikit-learn `make_column_selector`, que nos permite seleccionar columnas según su tipo de datos. Ahora ilustramos cómo usar esta función auxiliar.

In [5]:
from sklearn.compose import make_column_selector as selector

categorical_columns_selector = selector(dtype_include=object)
categorical_columns = categorical_columns_selector(data)
categorical_columns

['workclass',
 'education',
 'marital.status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'native.country']

Aquí, creamos el selector pasando el tipo de datos a incluir; luego pasamos el conjunto de datos de entrada al objeto selector, que devolvió una lista de nombres de columnas que tienen el tipo de datos solicitado. Ahora podemos filtrar las columnas no deseadas:

In [6]:
data_categorical = data[categorical_columns]
data_categorical.head()

,workclass,education,marital.status,occupation,relationship,race,sex,native.country
0,?,HS-grad,Widowed,?,Not-in-family,White,Female,United-States
1,Private,HS-grad,Widowed,Exec-managerial,Not-in-family,White,Female,United-States
2,?,Some-college,Widowed,?,Unmarried,Black,Female,United-States
3,Private,7th-8th,Divorced,Machine-op-inspct,Unmarried,White,Female,United-States
4,Private,Some-college,Separated,Prof-specialty,Own-child,White,Female,United-States


In [7]:
print(f"Los datos contienen {data_categorical.shape[1]} características")

Los datos contienen 8 características


Ahora presentaremos diferentes estrategias para codificar datos categóricos en datos numéricos que puedan ser utilizados por un algoritmo de aprendizaje automático.

## Estrategias para codificar categorías

### Codificación de categorías ordinales

La estrategia más intuitiva es codificar cada categoría con un número diferente. El `OrdinalEncoder` transforma los datos de esta manera. Comenzamos codificando una sola columna para entender cómo funciona la codificación.

In [8]:
from sklearn.preprocessing import OrdinalEncoder

education_column = data_categorical[["education"]]

encoder = OrdinalEncoder().set_output(transform="pandas")
education_encoded = encoder.fit_transform(education_column)
education_encoded

,education
0,11.0
1,11.0
2,15.0
3,5.0
4,15.0
...,...
32556,15.0
32557,7.0
32558,11.0
32559,11.0


Vemos que cada categoría en `"education"` ha sido reemplazada por un valor numérico. Podríamos verificar la correspondencia entre las categorías y los valores numéricos comprobando el atributo ajustado `categories_`.

In [9]:
encoder.categories_

[array(['10th', '11th', '12th', '1st-4th', '5th-6th', '7th-8th', '9th',
        'Assoc-acdm', 'Assoc-voc', 'Bachelors', 'Doctorate', 'HS-grad',
        'Masters', 'Preschool', 'Prof-school', 'Some-college'],
       dtype=object)]

Ahora, podemos verificar la codificación aplicada a todas las características categóricas.

In [10]:
data_encoded = encoder.fit_transform(data_categorical)
data_encoded[:5]

,workclass,education,marital.status,occupation,relationship,race,sex,native.country
0,0.0,11.0,6.0,0.0,1.0,4.0,0.0,39.0
1,4.0,11.0,6.0,4.0,1.0,4.0,0.0,39.0
2,0.0,15.0,6.0,0.0,4.0,2.0,0.0,39.0
3,4.0,5.0,0.0,7.0,4.0,4.0,0.0,39.0
4,4.0,15.0,5.0,10.0,3.0,4.0,0.0,39.0


In [11]:
print(f"Los datos codificados contienen {data_encoded.shape[1]} características")

Los datos codificados contienen 8 características


Vemos que las categorías han sido codificadas de forma independiente para cada característica (columna). También notamos que el número de características antes y después de la codificación es el mismo.

Sin embargo, hay que tener cuidado al aplicar esta estrategia de codificación: usar esta representación numérica hace que los modelos predictivos interpreten que los valores están ordenados (por ejemplo, 0 < 1 < 2 < 3...).

Por defecto, `OrdinalEncoder` utiliza una estrategia lexicográfica para asignar las etiquetas de categorías de cadena a enteros. Esta estrategia es arbitraria y, a menudo, carece de significado. Por ejemplo, supongamos que el conjunto de datos tiene una variable categórica llamada `"size"` con categorías como "S", "M", "L", "XL". Nos gustaría que la representación numérica respete el significado de los tamaños, asignándolos a enteros crecientes como `0, 1, 2, 3`. Sin embargo, la estrategia lexicográfica utilizada por defecto asignaría las etiquetas "S", "M", "L", "XL" a 2, 1, 0, 3, siguiendo el orden alfabético.

La clase `OrdinalEncoder` acepta un argumento de constructor `categories` para pasar explícitamente las categorías en el orden esperado. Puedes encontrar más información en la [documentación de scikit-learn](https://scikit-learn.org/stable/modules/preprocessing.html#encoding-categorical-features) si es necesario.

Si una variable categórica no tiene un orden significativo, esta codificación podría llevar a conclusiones erróneas en los modelos estadísticos posteriores, y deberías considerar el uso de codificación one-hot en su lugar (ver más abajo).

### Codificación de categorías nominales (sin asumir ningún orden)

`OneHotEncoder` es un codificador alternativo que evita que los modelos posteriores asuman un orden falso en las categorías. Para una característica dada, crea tantas columnas nuevas como categorías posibles. Para una muestra dada, el valor de la columna correspondiente a la categoría se establece en `1`, mientras que todas las columnas de las otras categorías se establecen en `0`.

Podemos codificar una sola característica (por ejemplo, `"education"`) para ilustrar cómo funciona la codificación.

In [12]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse_output=False).set_output(transform="pandas")
education_encoded = encoder.fit_transform(education_column)
education_encoded

,education_10th,education_11th,education_12th,education_1st-4th,education_5th-6th,education_7th-8th,education_9th,education_Assoc-acdm,education_Assoc-voc,education_Bachelors,education_Doctorate,education_HS-grad,education_Masters,education_Preschool,education_Prof-school,education_Some-college
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
32557,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32558,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
32559,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


<div class="admonition note alert alert-info">
<p class="first admonition-title" style="font-weight: bold;">Nota</p>
<p><tt class="docutils literal">sparse_output=False</tt> se utiliza en el <tt class="docutils literal">OneHotEncoder</tt> con fines didácticos, es decir, para facilitar la visualización de los datos.</p>
<p class="last">Las matrices dispersas son estructuras de datos eficientes cuando la mayoría de los elementos de tu matriz son ceros. No se cubrirán en detalle en este curso. Si deseas más información sobre ellas, puedes consultar <a class="reference external" href="https://scipy-lectures.org/advanced/scipy_sparse/introduction.html#why-sparse-matrices">este enlace</a>.</p>
</div>

Vemos que la codificación de una sola característica produce un dataframe lleno de ceros y unos. Cada categoría (valor único) se convierte en una columna; la codificación devuelve, para cada muestra, un 1 que especifica a qué categoría pertenece.

Apliquemos esta codificación al conjunto de datos completo.

In [13]:
print(f"Los datos contienen {data_categorical.shape[1]} características")
data_categorical.head()

Los datos contienen 8 características


,workclass,education,marital.status,occupation,relationship,race,sex,native.country
0,?,HS-grad,Widowed,?,Not-in-family,White,Female,United-States
1,Private,HS-grad,Widowed,Exec-managerial,Not-in-family,White,Female,United-States
2,?,Some-college,Widowed,?,Unmarried,Black,Female,United-States
3,Private,7th-8th,Divorced,Machine-op-inspct,Unmarried,White,Female,United-States
4,Private,Some-college,Separated,Prof-specialty,Own-child,White,Female,United-States


In [14]:
data_encoded = encoder.fit_transform(data_categorical)
data_encoded[:5]

,workclass_?,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,education_10th,...,native.country_Portugal,native.country_Puerto-Rico,native.country_Scotland,native.country_South,native.country_Taiwan,native.country_Thailand,native.country_Trinadad&Tobago,native.country_United-States,native.country_Vietnam,native.country_Yugoslavia
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [15]:
print(f"Los datos codificados con one.hot encoding contienen {data_encoded.shape[1]} características")

Los datos codificados con one.hot encoding contienen 102 características


Observa cómo se ha codificado la variable `"workclass"` en los primeros 3 registros y compáralo con su representación original en forma de cadena.

El número de características después de la codificación es más de 10 veces mayor que en los datos originales porque algunas variables, como `occupation` y `native.country`, tienen muchas categorías posibles.

### Elegir una estrategia de codificación

Elegir una estrategia de codificación depende de los modelos subyacentes y del tipo de categorías (es decir, ordinales vs. nominales).

Nota

En general, OneHotEncoder es la estrategia de codificación utilizada cuando los modelos posteriores son modelos lineales, mientras que OrdinalEncoder suele ser una buena estrategia con modelos basados en árboles.

Usar un `OrdinalEncoder` produce categorías ordinales. Esto significa que hay un orden en las categorías resultantes (por ejemplo, `0 < 1 < 2`). El impacto de violar esta suposición de orden depende mucho de los modelos posteriores. Los modelos lineales se verían afectados por categorías mal ordenadas, mientras que los modelos basados en árboles no.

Aún puedes usar un `OrdinalEncoder` con modelos lineales, pero debes asegurarte de que:
- las categorías originales (antes de la codificación) tengan un orden;
- las categorías codificadas sigan el mismo orden que las categorías originales.

El **próximo ejercicio** resalta el problema de usar incorrectamente `OrdinalEncoder` con un modelo lineal.

La codificación one-hot de variables categóricas con alta cardinalidad puede causar ineficiencia computacional en modelos basados en árboles. Por ello, no se recomienda usar `OneHotEncoder` en tales casos, incluso si las categorías originales no tienen un orden determinado.

## Evaluar nuestro pipeline predictivo

Ahora podemos integrar este codificador dentro de un pipeline de aprendizaje automático, como hicimos con los datos numéricos: entrenemos un clasificador lineal con los datos codificados y comprobemos el rendimiento generalizado de este pipeline de aprendizaje automático utilizando validación cruzada.

Antes de crear el pipeline, debemos detenernos en la columna `native.country`. Recordemos algunas estadísticas relacionadas con esta columna.

In [16]:
data["native.country"].value_counts()

,count
native.country,
United-States,29170
Mexico,643
?,583
Philippines,198
Germany,137
Canada,121
Puerto-Rico,114
El-Salvador,106
India,100


Vemos que la categoría `"Holand-Netherlands"` ocurre raramente. Esto será un problema durante la validación cruzada: si la muestra termina en el conjunto de prueba durante la división, el clasificador no habrá visto la categoría durante el entrenamiento y no podrá codificarla.

En scikit-learn, hay algunas soluciones posibles para evitar este problema:

* Listar todas las categorías posibles y proporcionarlas al codificador a través del argumento `categories` en lugar de permitir que el estimador las determine automáticamente a partir de los datos de entrenamiento al llamar a fit.
* Establecer el parámetro `handle_unknown="ignore"`, es decir, si se encuentra una categoría desconocida durante la transformación, las columnas resultantes de codificación one-hot para esta característica serán todas ceros.
* Ajustar el parámetro `min_frequency` para agrupar las categorías más raras observadas en los datos de entrenamiento en una sola característica codificada one-hot. Si habilitas esta opción, también puedes establecer `handle_unknown="infrequent_if_exist"` para codificar las categorías desconocidas (categorías solo observadas en el momento de la predicción) como unos en esa última columna.

En este notebook solo exploramos la segunda opción, es decir, `OneHotEncoder(handle_unknown="ignore")`. Siéntete libre de evaluar las alternativas por tu cuenta, por ejemplo, utilizando un notebook de pruebas.

Consejo

Ten en cuenta que el OrdinalEncoder también expone un parámetro llamado `handle_unknown`.
Se puede configurar en `use_encoded_value`. Si se eliges esta opción, puedes definir un valor fijo que se asigna a todas las categorías desconocidas durante la transformación con transform.
Por ejemplo, `OrdinalEncoder(handle_unknown="use_encoded_value",  class="pre">unknown_value=-1)` establecería todos los valores encontrados durante transform a -1 que no formen parte de los datos encontrados durante la llamada a fit. Vas a usar estos parámetros en el siguiente ejercicio.

Ahora podemos crear nuestro pipeline de aprendizaje automático para los datos codificados con OrdinalEncoder.

In [17]:
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression

modelOrdinal = make_pipeline(
    OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1,), LogisticRegression(max_iter=500)
)

Vemos el rendimiento de generalización del modelo utilizando solo las columnas categóricas.

In [18]:
from sklearn.model_selection import cross_validate
cv_results_ordinal = cross_validate(modelOrdinal, data_categorical, target)
cv_results_ordinal

{'fit_time': array([0.30248356, 0.35487151, 0.33186483, 0.37966323, 0.34736633]),
 'score_time': array([0.03930712, 0.03611732, 0.03515959, 0.03529286, 0.03597331]),
 'test_score': array([0.74834945, 0.75261057, 0.75429975, 0.75399263, 0.75583538])}

In [19]:
scores = cv_results_ordinal["test_score"]
print(f"The accuracy is: {scores.mean():.3f} ± {scores.std():.3f}")

The accuracy is: 0.753 ± 0.003


Ahora podemos crear nuestro pipeline de aprendizaje automático para los datos codificados con OneHotEncoder.

In [20]:
model = make_pipeline(
    OneHotEncoder(handle_unknown="ignore"), LogisticRegression(max_iter=500)
)

Vemos el rendimiento de generalización del modelo utilizando solo las columnas categóricas.

In [21]:
cv_results = cross_validate(model, data_categorical, target)
cv_results

{'fit_time': array([0.24181938, 0.28656602, 0.37635303, 0.76121664, 0.46939468]),
 'score_time': array([0.03991365, 0.03646207, 0.03695178, 0.06048107, 0.03666329]),
 'test_score': array([0.83602027, 0.82309582, 0.8252457 , 0.83845209, 0.82939189])}

In [22]:
scores = cv_results["test_score"]
print(f"The accuracy is: {scores.mean():.3f} ± {scores.std():.3f}")

The accuracy is: 0.830 ± 0.006


Como puedes ver, la representación OneHotEncoding es ligeramente mejor en accuracy que OrdinalEncoding ya que en modelos lineales se recomienda OneHotEncodign